In [1]:
#Originally used "!pip install pycbc lalsuite ligo-common" and "!pip install gwpy" but something about installing both pycbc and gwpy 
#causes colab to give errors in compatibilities, as well as, "missing" packages that asks you to restart runtime; dont restart runtime, just restart cell of pycbc
#installing an older version of gwpy or else a compatibility error happens

! pip install -q 'lalsuite==6.66' 'gwpy==1.0.1'

     |████████████████████████████████| 28.5 MB 68 kB/s 
     |████████████████████████████████| 1.4 MB 22.1 MB/s 
     |████████████████████████████████| 52 kB 559 kB/s 
     |████████████████████████████████| 1.6 MB 16.1 MB/s 
     |████████████████████████████████| 54 kB 1.7 MB/s 
     |████████████████████████████████| 3.2 MB 27.6 MB/s 


In [2]:
#running cell that imports gwpy near the top so if there's any errors, there won't be time wasted restarting the runtime and running cells again

import gwpy
from gwpy.timeseries import TimeSeries

In [3]:
#clone github repo of bajes

! git clone https://github.com/matteobreschi/bajes.git

Cloning into 'bajes'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 223 (delta 30), reused 223 (delta 30), pack-reused 0
Receiving objects: 100% (223/223), 11.93 MiB | 16.94 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [ ]:
#installs bajes repo to your content files on google colab, if using linux itll have its own directory

%cd bajes
! ls
! python setup.py install

In [ ]:
#necessary to run "TEOBResumS_NRPM" approx but not necessary if using "NRPM" approx
#however, if using "NRPM", lambda tides must >= 400 in it's paramaters in the dictionary cell a few cells down

! git clone https://RoxGamba@bitbucket.org/eob_ihes/teobresums.git

In [ ]:
#TEOBResumS wont install properly unless installing the build-essentials and lib*

! apt update
! apt install build-essential
! apt-get install -y libconfig-dev
!apt-get install libgsl-dev
% cd teobresums/Python/
!python TEOBResumSWrap_setup.py install

In [7]:
#Importing some packages

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#depending on how bajes uploads on google colab or locally, the path for bajes might be different. 
#instead of "bajes.bajes.obs..." it might just be "bajes.obs..."

from bajes.bajes.obs.gw import Series
from bajes.bajes.obs.gw import Noise
from bajes.bajes.obs.gw.utils import read_asd
import csv

In [8]:
#import google drive so files we are making wont disappear after runtime is over. 
#this is unnecessary if not using google colab

from google.colab import drive 
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


In [ ]:
#As of this moment there are 11 known events detected from the advanced gravitational-wave detectors
#we can access the data from the known events from the gwpy package and the time of event currently listed is from GW150914
#Waveform approximants(bottom of this cell) can be run independently from the event data, however, hard coding/replacing "'time'", "hpc.plus", etc and 
#other adjustments of the code made it difficult for me to change the central time to equal zero so accessing the data was easier

#---------------------------------------------------------------------

#should the need to not use the data from events occur, 
#comment out everything above "#set the data properties", comment out "series", and add: 

#dT     = 1/srate
#df = 1/seglen
#f = np.arange(0, srate/2, df)
#wave  = Waveform(f, srate, seglen, 'NRPM')

#then in the next cell, comment out "hpc" and add:

#hp, hc = wave.compute_hphc(params)
# t = np.array(range(0, len(hp)))*dT

#----------------------------------------------------------------------

time_of_event = 1126259462.4

post_trigger_duration =4
duration = 8
analysis_start = time_of_event + post_trigger_duration - duration

# Use gwpy to fetch the open data
H1_analysis_data = TimeSeries.fetch_open_data(
    "H1", analysis_start, analysis_start + duration, sample_rate=4096, cache=True)

t = H1_analysis_data.times
strain = H1_analysis_data.value

# set the data properties 
seglen = 8           # duration of the segment [s]
srate  = 4096  * 16       # sampling rate [Hz]
t_gps  = 0  # central value of GPS time
f_max  = 1024 * 4
f_min  = 20 

series = Series('time', strain, seglen=seglen, srate=srate, t_gps=t_gps, f_min=f_min, f_max=f_max)
print('here')

from bajes.bajes.obs.gw import Waveform

wave  = Waveform(series.freqs, srate, seglen, 'TEOBResumS_NRPM') #NRPM for postmergers

In [ ]:
!pip install pycbc
import pylab

from pycbc import waveform

In [47]:
from pycbc.waveform import get_td_waveform
from pycbc.types import TimeSeries, FrequencySeries, Array, float32, float64, complex_same_precision_as, real_same_precision_as

In [66]:
#since we want a lot of waveforms, we run our this cell with a for loop
#change the range number based on how many sims you want

for num in range(1):

  #----------------------------------------------------------------------------------------

  #This part of code is from Dr. Phillip Landry and what it does is deternmine a set of masses and 
  #lambda values for binary neutron stars that agree with each other

  from scipy.interpolate import interpolate as interp

  eosdir = '/content/drive/MyDrive/Colab_Notebooks/macro'
  numeos = 2396


  def get_mmax(eos_id):
    eospath = eosdir+"/macro-spec_%dcr.csv" % eos_id
    eos_data = np.genfromtxt(eospath, names=True, delimiter=",")
    marray =  eos_data["M"]
    mmax_pos   = np.argmax(marray)
    mmax = marray[mmax_pos]
    return mmax

  def get_lambda_from_masses(m1, m2, eos_id):

    eospath = eosdir+"/macro-spec_%dcr.csv" % eos_id
    eos_data = np.genfromtxt(eospath, names=True, delimiter=",")
    marray, larray =  eos_data["M"], eos_data["Lambda"]
    mmax_pos   = np.argmax(marray)
    marray, larray = marray[0:mmax_pos+1], larray[0:mmax_pos+1]
    mmax = marray[mmax_pos]

    assert m1 <= mmax and m2 <= mmax, "Error: one or both masses too heavy for a NS!"

    f_lambda = interp.interp1d(marray, larray, fill_value=0, bounds_error=False)
    lambda1, lambda2 = f_lambda(m1), f_lambda(m2)

    assert lambda2 >= lambda1, "Error: Lambda1 is larger than Lambda2 for common EOS!"

    return lambda1, lambda2

  #Dr. Landry's code requires a file that contains different equations of state(eos). Normally this data
  #is on the LIGO supercomputers, however, since I am not using colab, I had to upload the file to drive

  #-------------------------------------------------------------------------------------------------------------
  
  import math
  import random

  #picks a random eos file and gets mass  

  eos_id = np.random.choice(range(numeos), 1)
  mmax = get_mmax(eos_id)

  #randomly picks masses between 1 and the returned mmax 

  m1 = random.uniform(1, mmax) 
  m2 = random.uniform(1, mmax)

  # m1 has to be bigger than m2 for the proper mass ratio 
  # q calculates the mass ratio 

  if m1 > m2:
    chirp_mass = (math.pow((m1 * m2), (3/5))) / (math.pow((m1 + m2), (1/5))) 
    q = m1 / m2 
  elif m1 < m2:
    while m1 < m2:
      m1 = random.uniform(1, mmax)
      m2 = random.uniform(1, mmax)
  chirp_mass = (math.pow((m1 * m2), (3/5))) / (math.pow((m1 + m2), (1/5)))
  q = m1 / m2

  #gets the lambdas from the chosen masses

  lambda1, lambda2 = get_lambda_from_masses(m1, m2, eos_id)

  #dictionary parameters for making the timeseries 

  params = {'mchirp'       : chirp_mass,    # chirp mass [solar masses] 
              'q'          : q,      # mass ratio 
              's1x'        : 0.,      # primary spin parameter, x component
              's1y'        : 0.,      # primary spin parameter, y component
              's1z'        : 0.,      # primary spin parameter, z component
              's2x'        : 0.,      # secondary spin parameter, x component
              's2y'        : 0.,      # secondary spin parameter, y component
              's2z'        : 0.,      # secondary spin parameter, z component
              'lambda1'    : lambda1,    # primary tidal parameter 
              'lambda2'    : lambda2,    # secondary tidal parameter
              'distance'   : 100.8114416513031,    # distance [Mpc]   
              'iota'       : np.pi,   # inclination [rad]   
              'ra'         : 0.,     # right ascension [rad]
              'dec'        : 0.,   # declination [rad]
              'psi'        : 0.,      # polarization angle [rad]
              'time_shift' : 0.419,   # time shift from GPS time [s]
              'phi_ref'    : 0.,      # phase shift [rad]
              'f_min'      : 20.,     # minimum frequency [Hz]
              'srate'      : srate,   # sampling rate [Hz]
              'seglen'     : seglen,  # segment duration [s] 
              'tukey'      : 0.1,     # parameter for tukey window
              't_gps'      : t_gps,   # GPS trigger time
              'lmax'       : 0.,
              'eccentricity' : 0.
             }  

  #strain of time series

  hp, hc = wave.compute_hphc(params)

  #turn hp and hc into a separate time series so we can get its amplitude and phase
  ts_hp = TimeSeries(hp, delta_t=1/(4096*16))
  ts_hc = TimeSeries(hc, delta_t=1/(4096*16))
  
  #use pycbc to get amp and phase
  amp = waveform.utils.amplitude_from_polarizations(ts_hp, ts_hc)
  phase = waveform.utils.phase_from_polarizations(ts_hp, ts_hc)

  #additional keys for our metadata

  new_params = params #put params in new dictionary
  new_params["m1"] = m1 #add m1 to new dictionary
  new_params["m2"] = m2 #add m2 to new dictionary
  new_params["mtot"] = round((m1 + m2), 1) #change mtot from params into a float with only 1 decimal point


 #the very first run of this cell, "next_simulation = 1"
 #if eos code throws an error while being iterated, change the next_simulation variable to the last simulation number + 1 so if 
 #last simulation file name number is "S00053" then before you restart the cell, put "next_simulation = 54"
 #this was made when a version of this kernal kept getting assert errors, but is no longer really necessary; however,
 #it is useful for having the file name start at 1 instead of 0 and that's why i kept it

  next_simulation = 1
  if num == 0:
    new_num = num + next_simulation
  else:
    new_num = new_num + 1

  #changes the file name for each sim run so it looks nicer imo

  if new_num <= 9:
    leading_zeros = "S0000"
  elif new_num <= 99:
    leading_zeros = "S000"
  elif new_num <= 999:
    leading_zeros = "S00"
  elif new_num <= 9999:
    leading_zeros = "S0"
  else:
    leading_zeros = "S"


  #assigning arguments into variables so it can be changed easily
  #new_params["file name"] changes the dictionary key to have the associated time series csv filename within the metadata
  new_params["file name"] = f"{leading_zeros}{new_num}.csv"

  #text_file is the path for the time series files to save in
  text_file = f'/content/drive/MyDrive/Colab_Notebooks/{leading_zeros}{new_num}.csv'



  #separates the time series into two columns so it can be in a dataframe

  x = series.times
  ts_df= pd.DataFrame({'time': x, 'amplitude': amp, 'h_plus': hp, 'h_cross': hc, 'phase': phase})



  #deletes rows from original dataframe so the time series csv will only be between -0.6 and +0.6 seconds with srate = 4096
  #ts_df.drop(ts_df.index[0:16138], 0, inplace = True)
  #ts_df.drop(ts_df.index[493:], 0, inplace = True)

  #deletes rows from original dataframe so the time series csv will only be between -0.6 and +0.6 seconds with srate = 4096 * 16
  #however, since the time series needs to be within a power of 2, we must adjust the dataframe to be within 8192 rows
  #dropping the first 258049 and keeps only 8192 rows to land the time series between -0.6... and +0.6... seconds
  ts_df.drop(ts_df.index[0:258049], 0, inplace = True)
  ts_df.drop(ts_df.index[8192:], 0, inplace = True)


  #formats dataframe
  
  ts_df.to_csv(text_file, sep= '\t', header = None, index = False) 




  #creates original metadata file then appends the subsequent iterations
  if new_num == 1:
    df = pd.DataFrame.from_dict([new_params])
    df.to_csv('/content/drive/MyDrive/Colab_Notebooks/METADATA_FILENAME_PRACTICE.csv', sep= '\t', index = False)
  else: 
    from csv import DictWriter

    def append_dict_as_row(file_name, dict_of_elem, field_names):
      # Open file in append mode
        with open(file_name, 'a+', newline='') as write_obj:
          # Create a writer object from csv module
            dict_writer = DictWriter(write_obj, delimiter = '\t', fieldnames=field_names)
            # Add dictionary as row in the csv
            dict_writer.writerow(dict_of_elem)

    field_names = ['mchirp', 'q', 's1x', 's1y', 's1z', 's2x', 's2y', 's2z', 
               'lambda1', 'lambda2', 'distance','iota', 'ra', 'dec', 'psi','time_shift',
               'phi_ref', 'f_min', 'srate', 'seglen', 'tukey', 't_gps', 'lmax', 'eccentricity', 'm1', 'm2', 'mtot', 'file name'
               ]  
    # Adds each simulation as a row in the metadata csv file
    row_dict = new_params
    append_dict_as_row('/content/drive/MyDrive/Colab_Notebooks/METADATA_FILENAME_PRACTICE.csv', row_dict, field_names)


